In [143]:
import pandas as pd
import re

complex_areas = pd.read_csv('PG_GET_FULL_ALL_202405171656.csv', low_memory=False)

complex_areas['startSaleDate'] = pd.to_datetime(complex_areas['startSaleDate'], errors='coerce')

filtered_complex_areas = complex_areas[complex_areas['startSaleDate'].dt.year.isin([2023, 2024])]

exclude_types = ['Коттедж', 'Пентхаус','Сплитхаус', 'Квадрохаус', 'Таунхаус', 'Дуплекс', 'Участок', 'Офис', 'Гараж', 'Цоколь']
filtered_complex_areas = filtered_complex_areas[~filtered_complex_areas['PomeshcheniyeType'].isin(exclude_types)]

In [144]:
filtered_complex_areas['PomeshcheniyeType'].unique()

array(['Квартира', 'Кладовка', 'Паркинг', 'Наземный паркинг'],
      dtype=object)

In [145]:
def extract_complex_name(name):
    match = re.match(r'^(.*?)\s[А-Яа-я]*$', name)
    return match.group(1).strip() if match else name.strip()

filtered_complex_areas['ComplexName'] = filtered_complex_areas['ObjectName'].apply(extract_complex_name)

In [146]:
df = filtered_complex_areas.copy()

In [147]:
df.ComplexName = df.ObjectName.str.lower().str.replace('паркинг','').str.replace('очередь','').str.replace('оч','').str.replace('кладовки','')\
.str.replace('кладовые','').str.replace('вп','').str.replace('отделка','').str.replace('подвал','').str.replace('подъезд','').str.replace('надземный','')\
.str.replace('отдельностоящее коммерческое','').str.replace('отдельностоящие','').str.replace('кладовка','').str.replace('и офисы','').str.replace('блоки','').str.replace('и цоколи','')\
.str.replace('(','').str.replace(')','').str.replace('подземный ','')

In [148]:
df.ComplexName = df.ComplexName.str.replace('-',' ').str.replace(' ', '')

In [150]:
df.query("ComplexName.str.startswith('ұлы') and  not ObjectName.str.contains('оч')", engine='python').head(2)

,ObjectGUID,Data,DivisionName,DivisionGUID,ObjectName,City,Class,Code,Location,Longtitude,...,ProcentVznosa,Aktsiya,AktsiyaGUID,SkidkaProcent,SkidkaNachalnikaOPProcent,ObjectNameOrig,startSaleDate,deadline,dateDDU,ComplexName
3389,c365016c-b8f7-11ed-a826-001dd8b72708,2024-05-17T06:25:30,Дивизион Шымкент,7d8eaf2f-6606-11eb-aa99-001dd8b721b9,Ұлы дала паркинг - 1 - Паркинг,Шымкент,Бизнес,1,"г.Шымкент, пр. Назарбаева и ул. Байтерекова",69.64046,...,NaN,NaN,NaN,NaN,NaN,?лы дала паркинг - 1 - Паркинг,2023-03-03,2025-03-29T00:00:00,2023-08-31T00:00:00,ұлыдала1
3390,c365016c-b8f7-11ed-a826-001dd8b72708,2024-05-17T06:25:30,Дивизион Шымкент,7d8eaf2f-6606-11eb-aa99-001dd8b721b9,Ұлы дала паркинг - 1 - Паркинг,Шымкент,Бизнес,2,"г.Шымкент, пр. Назарбаева и ул. Байтерекова",69.64046,...,NaN,NaN,NaN,NaN,NaN,?лы дала паркинг - 1 - Паркинг,2023-03-03,2025-03-29T00:00:00,2023-08-31T00:00:00,ұлыдала1


In [151]:
df[df.ComplexName.str.startswith('ұлыдал')].ComplexName.unique()

array(['ұлыдала1', 'ұлыдала2', 'ұлыдала'], dtype=object)

In [105]:
len(df.ComplexName.unique())

239

In [152]:
len(df.ComplexName.str.replace(' ', '').unique())

187

In [153]:
df.ComplexName.str.replace(' ', '').unique()

array(['grandturanкомфорт41', 'sadobusiness1', 'verdi1.', 'lifetown2',
       'arenacity.lifeабизнес', 'capitalparkemotions1',
       'dreamcityecoaстандарт', 'поколениебизнес2', 'nrgzamon2.2',
       'alapark31', 'amir3', 'alapark4', 'grandparkqasqasu2',
       'greenline.headlinerexclusive1', 'mod.comfort1', 'nrgjomiy1',
       'sezimqala.örken', 'arenaparkcomfort4', 'armanqala31',
       'jetisu.aqsu4', 'capitalpark.light3b2', 'lifetown12',
       'mod.comfort2', 'nrgjomiy1.2', 'sapa4lifecomfort11', 'turantower',
       'verdi1', 'клубныйдом44', 'mod.standard21', 'altynzaman12',
       'arenacity2', 'altynzaman11', 'arenacity.sportaстандарт',
       'gardenview2', 'aisar', 'mod.standard12', 'capitalpark.flowers5c1',
       'botanikasaroyi1', 'nrgyangibaxt1', 'nexpoclassic12',
       'arenaparkcomfort3', 'asylbesterek', 'atamuracomfort11',
       'atamuracomfort12', 'aulet', 'capitalpark.flowers5a',
       'capitalpark.flowers5c2', 'capitalpark.flowers5с1',
       'dendroparkbusiness

In [159]:
only_kv = df[df['PomeshcheniyeType'].str.startswith('Квартира')]

In [163]:
not_kv =df[~df['PomeshcheniyeType'].str.startswith('Квартира')]

In [161]:
kv_area = only_kv.groupby('ComplexName')['Area'].sum().reset_index()

In [162]:
kv_area

,ComplexName,Area
0,4youshymkentbusiness11,16513.56
1,4youshymkentbusiness12,11518.04
2,4youshymkentbussiness13,1950.17
3,aisar2,15840.22
4,akbulakrivierawall–house12,3019.95
...,...,...
121,клубныйдом44,5395.23
122,поколениебизнес2,16064.80
123,поколениебизнес3,11582.80
124,ұлыдала1,17188.28


In [183]:
not_kv

,ObjectGUID,Data,DivisionName,DivisionGUID,ObjectName,City,Class,Code,Location,Longtitude,...,ProcentVznosa,Aktsiya,AktsiyaGUID,SkidkaProcent,SkidkaNachalnikaOPProcent,ObjectNameOrig,startSaleDate,deadline,dateDDU,ComplexName
6,0b0d1313-654c-4416-a16e-4890100e1938,2024-05-17T04:37:40,Дивизион Алматы 2,db5c31db-94e2-11ee-aaab-001dd8b71f55,Lifetown - 2 Кладовые,Алматы,Комфорт,16,"г. Алматы, р-н Алмалинский, ул. Торекулова, у...",76.92687,...,NaN,NaN,NaN,NaN,NaN,Lifetown - 2 Кладовые,2024-01-13,2024-09-07T00:00:00,2023-07-18T00:00:00,lifetown2
21,8a69d454-3c92-11ec-a81f-001dd8b72708,2024-05-17T03:08:21,Дивизион Алматы 1,d1e0d53a-94e2-11ee-aaab-001dd8b71f55,ALA Park - 3-1 Кладовки,Алматы,Комфорт,28(442),"г. Алматы, р-н Бостандыкский, ул. Сатпаева, уч...",76.88446,...,NaN,NaN,NaN,NaN,NaN,ALA Park - 3-1 Кладовки,2023-07-29,2023-10-31T00:00:00,2022-02-28T00:00:00,alapark31
22,65728319-f214-11ec-a825-001dd8b72708,2024-05-17T03:10:36,Дивизион Алматы 2,db5c31db-94e2-11ee-aaab-001dd8b71f55,AMIR 3 - Паркинг,Алматы,Бизнес+,8(157),"город Алматы, Бостандыкский район, улица Розыб...",76.89538,...,NaN,NaN,NaN,NaN,NaN,AMIR 3 - Паркинг,2023-05-27,2024-03-03T00:00:00,2022-04-29T00:00:00,amir3
23,501e4c82-8ea5-4eeb-8763-42908c97ff66,2024-05-17T03:15:46,Дивизион Алматы 1,d1e0d53a-94e2-11ee-aaab-001dd8b71f55,Ala Park 4 - паркинг,NaN,Комфорт,113(252),"г. Алматы, р-н Бостандыкский, ул. Сатпаева, уч...",76.88446,...,NaN,NaN,NaN,NaN,NaN,Ala Park 4 - паркинг,2024-04-26,2024-09-28T00:00:00,0001-01-01T00:00:00,alapark4
26,efb5c6be-a12b-11ed-a826-001dd8b72708,2024-05-17T04:22:38,Дивизион Астана 1 Дирекция 3,9676817c-af9d-11ee-aab0-001dd8b721b9,GreenLine.Headliner Exclusive - 1 - Паркинг,Астана,Бизнес+,28,"РК, Астана г., район Есиль, район пересечения ...",71.39858,...,NaN,NaN,NaN,NaN,NaN,GreenLine.Headliner Exclusive - 1 - Паркинг,2023-03-20,2024-07-20T00:00:00,2023-08-16T00:00:00,greenline.headlinerexclusive1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184362,a2d20eb4-b1c8-11ed-a82b-001dd8b726aa,2024-05-17T04:40:27,Дивизион Астана 2 Дирекция 1,53c9058b-16d4-11e3-936a-002590952079,MoD.Comfort - 1 - Паркинг,Астана,Комфорт,70,"РК, Астана г., район Есиль, улица Е809, участо...",71.43147,...,NaN,NaN,NaN,NaN,NaN,MoD.Comfort - 1 - Паркинг,2023-10-02,2024-07-27T00:00:00,2023-07-20T00:00:00,mod.comfort1
184385,cdad405a-f48b-11ed-a82e-001dd8b726aa,2024-05-17T04:43:04,NRG,e2937123-506e-11ee-aaad-001dd8b721b9,NRG Jomiy кладовые - 1,Ташкент,Бизнес,27,"Республика Узбекистан, г,Ташкент Алмазарский р...",69.25746,...,30.0,NaN,NaN,6.0,0.0,NRG Jomiy кладовые - 1,2023-07-07,2024-11-23T00:00:00,0001-01-01T00:00:00,nrgjomiy1
184394,ae153c34-3b56-45ed-849e-5a326efe312e,2024-05-17T05:33:12,Дивизион Алматы 1,d1e0d53a-94e2-11ee-aaab-001dd8b71f55,Verdi 1 - кладовые,NaN,Бизнес,49,"г. Алматы, р-н Бостандыкский, ул. Егизбаева, у...",76.88591,...,NaN,NaN,NaN,NaN,NaN,Verdi 1 - кладовые,2024-05-06,2025-08-30T00:00:00,2024-04-01T00:00:00,verdi1
184400,6af99297-954c-11ed-a82b-001dd8b726aa,2024-05-17T04:37:11,Дивизион Алматы 2,db5c31db-94e2-11ee-aaab-001dd8b71f55,Lifetown - 1-2 Паркинг,Алматы,Комфорт,141(242),"город Алматы, Алмалинский район, улица Торекул...",76.92687,...,NaN,NaN,NaN,NaN,NaN,Lifetown - 1-2 Паркинг,2023-09-27,2024-01-14T00:00:00,2023-02-14T00:00:00,lifetown12


In [ ]:
sum_plus = 0


In [181]:
sum_plus = 0
not_in = 0
for index, object_name in not_kv.loc[:,'ObjectName'].items():
    if kv_area.ComplexName.str.contains(object_name.split()[0].lower()).any():
        sum_plus +=  not_kv.loc[index,'Area']
    else:
        not_in+=1

In [182]:
not_in

839

In [179]:
sum_plus/126

954.8291269841326

In [180]:
954+14535

15489

In [178]:
areas = filtered_complex_areas.groupby('ComplexName')['Area'].sum().reset_index()
areas.columns = ['Complex', 'TotalArea']
areas['TotalArea'].mean()

8172.168583333333

In [115]:
areas['TotalArea'].count()

209

In [124]:
dd = filtered_complex_areas[filtered_complex_areas['PomeshcheniyeType'].str.startswith('Квартира')]

In [125]:
dd

,ObjectGUID,Data,DivisionName,DivisionGUID,ObjectName,City,Class,Code,Location,Longtitude,...,Aktsiya,AktsiyaGUID,SkidkaProcent,SkidkaNachalnikaOPProcent,ObjectNameOrig,startSaleDate,deadline,dateDDU,ComplexName,Index
1,f0ad1f3e-86a6-11e9-a82b-00155d101622,2024-05-17T04:08:29,Дивизион Астана 1 Дирекция 1,d5a2f5b9-5069-11ee-aaad-001dd8b721b9,GRAND TURAN (Комфорт) - 4 - 1,Астана,Комфорт+,56,"РК, Астана г., район Есиль, район пересечения ...",71.40000,...,NaN,NaN,NaN,NaN,GRAND TURAN (Комфорт) - 4 - 1,2023-01-27,2024-08-17T00:00:00,2023-04-27T00:00:00,GRAND TURAN (Комфорт) - 4 -,1
3,6720b954-229a-4ccc-87f0-0c2d911c98fc,2024-05-17T05:21:39,BI Tashkent,d77c8004-506e-11ee-aaad-001dd8b721b9,Sado Business - 1,Ташкент,Бизнес,52,"Город Ташкент, Яшнобадский район, Махаллинский...",69.34363,...,NaN,NaN,NaN,NaN,Sado Business - 1,2024-03-26,2025-09-27T00:00:00,2024-07-25T00:00:00,Sado Business -,1
4,e56a8ee5-1aea-11ec-a829-001dd8b726aa,2024-05-17T05:32:57,Дивизион Алматы 1,d1e0d53a-94e2-11ee-aaab-001dd8b71f55,Verdi - 1 оч.,Алматы,Бизнес,17,"г. Алматы, р-н Бостандыкский, ул. Егизбаева, у...",76.88591,...,NaN,NaN,NaN,NaN,Verdi - 1 оч.,2024-02-24,2025-08-30T00:00:00,2024-04-01T00:00:00,Verdi - 1 оч.,
10,50e61628-5d1b-11ee-a82a-001dd8b72708,2024-05-17T03:19:11,Дивизион Алматы 1,d1e0d53a-94e2-11ee-aaab-001dd8b71f55,Arena City. Life А (бизнес),Алматы,Бизнес,14,"г.Алматы, р-н Алатауский, мкр. Акмаржан, уч.1",76.81511,...,NaN,NaN,NaN,NaN,Arena City. Life А (бизнес),2024-04-06,2025-11-29T00:00:00,2024-09-30T00:00:00,Arena City. Life А (бизнес),
11,1adff43b-19fb-11e7-97ac-b4b52f5405e7,2024-05-17T03:41:56,Дивизион Астана 2 Дирекция 1,53c9058b-16d4-11e3-936a-002590952079,Capital Park Emotions - 1,Астана,Комфорт,99,"РК, Астана г., район Есиль, район пересечения ...",71.44575,...,NaN,NaN,NaN,NaN,Capital Park Emotions - 1,2024-04-25,2025-09-13T00:00:00,2024-08-15T00:00:00,Capital Park Emotions -,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184392,62027a55-4f04-11e6-9a13-b4b52f5405e7,2024-05-17T04:11:46,Дивизион Астана 1 Дирекция 2,628a565f-4cb1-11ee-aaad-001dd8b721b9,Garden View-2,Астана,Бизнес+,14,"РК, Астана г., район Есиль, улица Букар Жырау,...",71.40981,...,NaN,NaN,NaN,NaN,Garden View-2,2023-04-27,2024-10-26T00:00:00,2023-05-03T00:00:00,Garden View-2,
184395,e56a8ee5-1aea-11ec-a829-001dd8b726aa,2024-05-17T05:32:57,Дивизион Алматы 1,d1e0d53a-94e2-11ee-aaab-001dd8b71f55,Verdi - 1 оч.,Алматы,Бизнес,16,"г. Алматы, р-н Бостандыкский, ул. Егизбаева, у...",76.88591,...,NaN,NaN,NaN,NaN,Verdi - 1 оч.,2024-02-24,2025-08-30T00:00:00,2024-04-01T00:00:00,Verdi - 1 оч.,
184399,a894458e-9b8d-11eb-a83f-00155d106fa9,2024-05-17T05:23:45,Дивизион Астана 2 Дирекция 2,a0370746-506b-11ee-aaad-001dd8b721b9,Sezim Qala. Örken,Астана,Стандарт,144,пр.Туран,71.39717,...,NaN,NaN,NaN,NaN,Sezim Qala. Orken,2024-06-12,2025-06-21T00:00:00,0001-01-01T00:00:00,Sezim Qala. Örken,
184403,ca17024d-644e-11ee-a82a-001dd8b72708,2024-05-17T04:52:20,NRG,e2937123-506e-11ee-aaad-001dd8b721b9,NRG Zamon - 2.2,Ташкент,Комфорт,255,NaN,69.35599,...,NaN,NaN,NaN,NaN,NRG Zamon - 2.2,2024-09-21,0001-01-01T00:00:00,0001-01-01T00:00:00,NRG Zamon - 2.2,


In [127]:
dd.groupby('ObjectName')['Area'].sum().describe()

count      126.000000
mean     14535.068889
std       6041.771342
min         70.030000
25%      10801.215000
50%      14972.970000
75%      17740.727500
max      32697.210000
Name: Area, dtype: float64